In [1]:
import random, math
from itertools import count

import torch
from torch import nn, optim
import torch.nn.functional as F

import gym

import matplotlib.pyplot as plt

from memory import ReplayMemory

<img src="ddpg.png" width="1006px" height="744px"></img>

In [90]:
class Actor(nn.Module):
    pass

class Critic(nn.Module):
    pass